# Send REST requests to the WeatherApp.API running in Aspire

In [6]:
// First set the base url.
var baseUrl = "https://localhost:7220";

## Healthcheck 💉

In [7]:
#!set --value @csharp:baseUrl --name baseUrl
GET {{baseUrl}}/

Request GET https://localhost:7220/ HTTP/1.1 Headers Name Value traceparent 00-5b5574261e729e130d8f6587071946e1-2e13a44daddcab01-00 Body (text/plain; charset=utf-8, 0 bytes) Response HTTP/1.1 200 OK (98.81 ms) Headers Name Value Date Tue, 22 Apr 2025 20:56:31 GMT Server Kestrel Transfer-Encoding chunked Content-Type text/plain; charset=utf-8 Content-Length 26 Body (text/plain; charset=utf-8, 26 bytes) WeatherApp API is running!

## GET Weather Forecast 🌦️🌈🔆

In [8]:
#!set --value @csharp:baseUrl --name baseUrl
GET {{baseUrl}}/v1/weather-forecast/Bristol/{{$datetime iso8601}}

Request GET https://localhost:7220/v1/weather-forecast/Bristol/2025-04-22T20:56:37.9333473Z HTTP/1.1 Headers Name Value traceparent 00-68225243c6737248fd9296ac0b4c0eb6-2b41ad3952ec7b6f-00 Body (application/json; charset=utf-8, 0 bytes) Response HTTP/1.1 200 OK (125.36 ms) Headers Name Value Date Tue, 22 Apr 2025 20:56:38 GMT Server Kestrel Transfer-Encoding chunked Content-Type application/json; charset=utf-8 Content-Length 175 Body (application/json; charset=utf-8, 175 bytes) {"requestedRegion":"Bristol","requestedDate":"2025-04-22T20:56:37.9333473Z","requestId":"a3cf0256-bc02-4977-a4f8-1752381cc3fb","temperature":32,"summary":"summary from cache"} requestedRegion "Bristol" requestedDate "2025-04-22T20:56:37.9333473Z" requestId "a3cf0256-bc02-4977-a4f8-1752381cc3fb" temperature 32 summary "summary from cache"

## POST Colected Weather Data 🌡️☔💨

In [15]:
// Generate a random reference...
string[] fruits = ["apple", "banana", "cherry", "date", "elderberry", "fig", "grape", "honeydew", "kiwi", "lemon", "mango", "nectarine", "orange", "papaya", "quince", "raspberry", "strawberry", "tangerine", "watermelon", "guava"];
var randomReference = $"{fruits[new Random().Next(fruits.Length)]}-{Random.Shared.Next(10, 99)}";
randomReference.Display();

mango-27

In [16]:
#!set --value @csharp:baseUrl --name baseUrl
#!set --value @csharp:randomReference --name randomReference

POST {{baseUrl}}/v1/collected-weather-data/Bristol/{{randomReference}}
Content-Type: application/json
x-request-id: {{$guid}}
    
{
    "points": [
        {
            "time": "{{$datetime iso8601}}",
            "windSpeedInMetersPerSecond": {{$randomInt 0 69}}.{{$randomInt 0 99}},
            "windDirection": "N",
            "temperatureReadingInDegreesCelcius": {{$randomInt -15 40}}.{{$randomInt 0 99}},
            "humidityReadingInPercent": {{$randomInt 20 100}}
        }
    ]
}

Request POST https://localhost:7220/v1/collected-weather-data/Bristol/mango-27 HTTP/1.1 Headers Name Value X-Request-ID 676f3fe7-1f5b-45fe-826d-a3242d285aa7 traceparent 00-b91475794963d4be854aa91668c51140-1a6ebd215084dc75-00 Content-Type application/json Content-Length 279 Body (application/json; charset=utf-8, 279 bytes) {
 "points": [
 {
 "time": "2025-04-23T21:29:27.0327092Z",
 "windSpeedInMetersPerSecond": 39.78,
 "windDirection": "N",
 "temperatureReadingInDegreesCelcius": 0.22,
 "humidityReadingInPercent": 78
 }
 ]
} Response HTTP/1.1 200 OK (3503.88 ms) Headers Name Value Date Wed, 23 Apr 2025 21:29:29 GMT Server Kestrel Transfer-Encoding chunked Content-Type application/json; charset=utf-8 Content-Length 75 Body (application/json; charset=utf-8, 75 bytes) {"requestId":"fab410f9-2916-44ce-b642-6c1585ac38ea","reference":"mango-27"} requestId "fab410f9-2916-44ce-b642-6c1585ac38ea" reference "mango-27"

<div style="display:block; height:40rem;"></div>

## GET Traces 🤖

In [17]:
using System.Net.Http;
using System.Text.Json;

public class TraceData
{
    public string Resource { get; set; } = string.Empty;
    public string Source { get; set; } = string.Empty;
    public string DisplayName { get; set; } = string.Empty;
    public string TraceId { get; set; } = string.Empty;
    public string SpanId { get; set; } = string.Empty;    
    public Dictionary<string, object> Tags { get; set; } = new();

    public override string ToString()
    {
        var thirdPart = string.Empty;
        if(DisplayName == "Domain Event Insertion")
            thirdPart = Tags["domain-event.eventclassName"].ToString().Split('.').LastOrDefault();

        if(DisplayName == "User Notication Sent")
            thirdPart = $"✉️ reference: {Tags["user-notification-event.reference"].ToString()}";

        if(DisplayName == "Outbox Item Insertion")
            thirdPart = $"📫 outbox id: {Tags["outbox-item.Id"].ToString()}";
        
            
        
        return $"{Resource.PadRight(30, '-')}> {DisplayName.PadRight(30, '-')}> {thirdPart}";
    } 
        
}

var client = new HttpClient() { BaseAddress = new Uri("https://localhost:7059") };
var response = await client.GetAsync("traces");
var responseJson = await response.Content.ReadAsStringAsync();
JsonSerializer.Deserialize<TraceData[]>(responseJson, JsonSerializerOptions.Web ).Display();

index value 0 WeatherApp.EventListener------> Outbox Item Insertion---------> 📫 outbox id: 20014 Resource WeatherApp.EventListener Source OutboxRepository DisplayName Outbox Item Insertion TraceId b91475794963d4be854aa91668c51140 SpanId 9e2d34f0f43279de Tags key value outbox-item.TypeName "UserNotificationEvent" outbox-item.Id 20014 1 WeatherApp.EventListener------> Domain Event Insertion--------> ModelingDataAccepted Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId b91475794963d4be854aa91668c51140 SpanId 774466cec9dc9c13 Tags key value domain-event.streamId "fab410f9-2916-44ce-b642-6c1585ac38ea" domain-event.version "5" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.ModelingDataAccepted" 2 WeatherApp.API----------------> Domain Event Insertion--------> SubmittedToModeling Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion TraceId b91475794963d4be854aa91668c51140 SpanId e784b1ea01ad7023 Tags key value domain-event.streamId "fab410f9-2916-44ce-b642-6c1585ac38ea" domain-event.version "4" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.SubmittedToModeling" 3 WeatherApp.API----------------> Domain Event Insertion--------> PendingContributorPaymentPosted Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion TraceId b91475794963d4be854aa91668c51140 SpanId 8e23cbfb193e591d Tags key value domain-event.streamId "fab410f9-2916-44ce-b642-6c1585ac38ea" domain-event.version "3" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.PendingContributorPaymentPosted" 4 WeatherApp.API----------------> Domain Event Insertion--------> LocationIdFound Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion TraceId b91475794963d4be854aa91668c51140 SpanId e518168a4eb3d3fd Tags key value domain-event.streamId "fab410f9-2916-44ce-b642-6c1585ac38ea" domain-event.version "2" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.LocationIdFound" 5 NotificationService-----------> User Notication Sent----------> ✉️ reference: mango-27 Resource NotificationService Source UserNotificationEventHandler DisplayName User Notication Sent TraceId b91475794963d4be854aa91668c51140 SpanId 99164a1d3abd6281 Tags key value user-notification-event.body "Dear user, your data has been submitted and included in our latest model" user-notification-event.reference "mango-27" user-notification-event.timestamp "2025-04-23T21:29:33.5882056+00:00" 6 WeatherApp.EventListener------> Domain Event Insertion--------> ModelUpdated Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId b91475794963d4be854aa91668c51140 SpanId c5faeb82c5c691a8 Tags key value domain-event.streamId "fab410f9-2916-44ce-b642-6c1585ac38ea" domain-event.version "8" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.ModelUpdated" 7 WeatherApp.EventListener------> Domain Event Insertion--------> SubmissionComplete Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId b91475794963d4be854aa91668c51140 SpanId d104eb2dc5848626 Tags key value domain-event.streamId "fab410f9-2916-44ce-b642-6c1585ac38ea" domain-event.version "7" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.SubmissionComplete" 8 WeatherApp.EventListener------> Domain Event Insertion--------> PendingContributorPaymentCommitted Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId b91475794963d4be854aa91668c51140 SpanId 3d2d1564db98c3cd Tags key value domain-event.streamId "fab410f9-2916-44ce-b642-6c1585ac38ea" domain-event.version "6" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.PendingContributorPaymentCommitted" 9 WeatherApp.API----------------> Domain Event Insertion--------> WeatherDataCollectionInitiated Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion